In [2]:
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.2 MB/s eta 0:00:00


In [7]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
text = "The sun is shining, and the birds are singing. It's a beautiful day for a picnic in the park. Families are gathered, enjoying sandwiches and laughter. Children play on the swings and run through the grass, their laughter filling the air. The day is perfect for outdoor fun and relaxation."
sentences = text.split(".")
tokens = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

In [15]:
with torch.no_grad():
    logits = model(tokens).logits

In [16]:
sentence_scores = []

for i, sentence in enumerate(text.split(".")):
    sentence_tokens = tokenizer.encode(sentence, return_tensors="pt", add_special_tokens=True)
    sentence_logits = model(sentence_tokens).logits
    sentence_scores.append(sentence_logits.sum())

In [17]:
sentence_scores

[tensor(-3096539.7500, grad_fn=<SumBackward0>),
 tensor(-3690265., grad_fn=<SumBackward0>),
 tensor(-2131019., grad_fn=<SumBackward0>),
 tensor(-4464213.5000, grad_fn=<SumBackward0>),
 tensor(-2463866., grad_fn=<SumBackward0>),
 tensor(-453318.2812, grad_fn=<SumBackward0>)]

In [18]:
# Set the number of sentences to include in the summary
num_sentences = 2

# Select the top sentences
top_sentences_indices = torch.argsort(torch.tensor(sentence_scores), descending=True)[:num_sentences]

# Generate the summary
summary = [sentences[i] for i in top_sentences_indices]

print(". ".join(summary))


.  Families are gathered, enjoying sandwiches and laughter
